In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.colors import LogNorm
from astropy.io import fits
import importlib

import statmorph
from statmorph.utils.image_diagnostics import make_figure

import utils.data as datutils
import utils.plots as plots
import utils.image as imutils
%matplotlib inline

In [ ]:
import warnings
from pandas.errors import ParserWarning

warnings.simplefilter(action='ignore', category=ParserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
plt.rcParams['xtick.labelsize'] = 13
plt.rcParams['ytick.labelsize'] = 13
plt.rcParams['axes.labelsize'] = 13
plt.rcParams["mathtext.fontset"] = "cm"
plt.rcParams["font.family"] = "serif"

# Main

In [ ]:
for fnum in np.arange(start=5, stop=10):
    print(str(fnum).center(10, '-'))
    bcg_file_0 = f'data/gadget3k_20/bcg_{str(fnum).zfill(4)}_125_0.fits'
    bcg_file_1 = f'data/gadget3k_20/bcg_{str(fnum).zfill(4)}_125_1.fits'
    bcg_file_2 = f'data/gadget3k_20/bcg_{str(fnum).zfill(4)}_125_2.fits'
    try:
        hdulist = fits.open(bcg_file_0)
    except: 
        continue
    bcg0 = hdulist[0].data
    hdulist = fits.open(bcg_file_1)
    bcg1 = hdulist[0].data
    hdulist = fits.open(bcg_file_2)
    bcg2 = hdulist[0].data
    fig, axs = plt.subplots(figsize=(16, 8), nrows=1, ncols=3)
    plots.display_img(image=bcg0, axs=axs[0])
    plots.display_img(image=bcg1, axs=axs[1])
    plots.display_img(image=bcg2, axs=axs[2])
    plt.show()

In [ ]:
bcg_file_0 = 'data/bcg_all/bcg_0001_125_0.fits'
bcg_file_1 = 'data/bcg_all/bcg_0001_125_1.fits'
bcg_file_2 = 'data/bcg_all/bcg_0001_125_2.fits'
hdulist = fits.open(bcg_file_0)
bcg0 = hdulist[0].data
hdulist = fits.open(bcg_file_1)
bcg1 = hdulist[0].data
hdulist = fits.open(bcg_file_2)
bcg2 = hdulist[0].data

In [ ]:
fig, axs = plt.subplots(figsize=(16, 8), nrows=1, ncols=3)
plots.display_img(image=bcg0, axs=axs[0])
plots.display_img(image=bcg1, axs=axs[1])
plots.display_img(image=bcg2, axs=axs[2])
plt.show()

In [ ]:
center = (len(bcg0[1])//2, len(bcg0[0])//2)
print(center)

In [ ]:
radius = 100
fig, axs = plt.subplots(nrows=1, ncols=1)
plots.display_img(image=bcg0,  axs=axs)
for scale in np.linspace(1, 10, num=5):
    circle = Circle((center[0], center[1]), radius*scale,
                    color='red', fill=False, linewidth=1.5)

    axs.add_patch(circle)
plt.show()

In [ ]:
morphs_list = []
for scale in np.linspace(1, 10, num=5):
    bcg_mask = imutils.circular_mask(image=bcg0, image_center=center, 
                                    radius=scale*radius)
    morph = statmorph.source_morphology(image=bcg0, segmap=bcg_mask, gain=2.25)
    morphs_list.append(morph[0]) 

bcg0_df = datutils.create_morph_df(
    morphs_list, name='results/bcg0.csv', save=True)
bcg0_df['radius'] = radius * np.linspace(1, 10, num=5)
bcg0_df.to_csv('results/bcg0.csv')

In [ ]:
morphs_list = []
for scale in np.linspace(1, 10, num=5):
    bcg_mask = imutils.circular_mask(image=bcg1, image_center=center,
                                        radius=scale*radius)
    morph = statmorph.source_morphology(image=bcg1, segmap=bcg_mask, gain=2.25)
    morphs_list.append(morph[0])

bcg1_df = datutils.create_morph_df(
    morphs_list, name='results/bcg1.csv', save=True)
bcg1_df['radius'] = radius * np.linspace(1, 10, num=5)
bcg1_df.to_csv('results/bcg1.csv')

In [ ]:
morphs_list = []
for scale in np.linspace(1, 10, num=5):
    bcg_mask = imutils.circular_mask(image=bcg2, image_center=center,
                                        radius=scale*radius)
    morph = statmorph.source_morphology(image=bcg2, segmap=bcg_mask, gain=2.25)
    morphs_list.append(morph[0])

bcg2_df = datutils.create_morph_df(
    morphs_list, name='results/bcg2.csv', save=True)
bcg2_df['radius'] = radius * np.linspace(1, 10, num=5)
bcg2_df.to_csv('results/bcg2.csv')

# Correlations

## DS & MAH

### First MM0(a)...

In [ ]:
mah_df_list = []
dir = 'data/gadgetx3k_20/AHF_History/'
for f in os.listdir(dir):
    file = dir + f
    mm0 = datutils.load_mah(file)
    mah_df_list.append(mm0)

In [ ]:
dsdf = pd.read_csv(
    'data/gadgetx3k_20/G3X_progenitors/DS_G3X_snap_128_center-cluster_progenitors.txt',
    sep=r'\s+', header=0)

int_columns = [0, 1, 2, 7]
column_names = dsdf.columns

for idx in range(len(column_names)):
    col_name = column_names[idx]
    if idx in int_columns:
        dsdf[col_name] = dsdf[col_name].astype(int)
    else:
        dsdf[col_name] = dsdf[col_name].astype(float)
dsdf.drop(columns=['rID[0]', 'Hid[1]', 'DS_200[2]', 'DS_500[7]'], inplace=True)

In [ ]:
corrs_list = []
z_list = []
mah_ds_dict = {}
for z in mah_df_list[0]['Redshift']:
    mah_df = pd.DataFrame(columns=['M/M0'])

    for region in range(20):
        row = mah_df_list[region].loc[mah_df_list[region]['Redshift'] == z, ['M/M0']]
        if not row.empty:
            mah_df = pd.concat([mah_df, row], ignore_index=True)

    if mah_df.shape[0] != dsdf[:20].shape[0]:
        # print(
        #     f"Skipping redshift {z}: mismatched rows {mah_df.shape[0]} vs {dsdf.shape[0]}")
        continue

    df = pd.concat([mah_df.reset_index(drop=True),
                   dsdf[:20].reset_index(drop=True)], axis=1)
    mah_ds_dict[z] = df
    corrs = df.corr(method='spearman')
    corrs_list.append(corrs)
    z_list.append(z)
    
z_array = np.array(z_list)
aexp = 1/(1+z_array)

In [ ]:
eta_200 = [df.loc['eta_200[3]', 'M/M0'] for df in corrs_list]
delta_200 = [df.loc['delta_200[4]', 'M/M0'] for df in corrs_list]
fm_200 = [df.loc['fm_200[5]', 'M/M0'] for df in corrs_list]
fm2_200 = [df.loc['fm2_200[6]', 'M/M0'] for df in corrs_list]
eta_500 = [df.loc['eta_500[8]', 'M/M0'] for df in corrs_list]
delta_500 = [df.loc['delta_500[9]', 'M/M0'] for df in corrs_list]
fm_500 = [df.loc['fm_500[10]', 'M/M0'] for df in corrs_list]
fm2_500 = [df.loc['fm2_500[11]', 'M/M0'] for df in corrs_list]

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.set_title('R200')
axs.plot(aexp, eta_200, label='eta')
axs.plot(aexp, delta_200, label='delta')
axs.plot(aexp, fm_200, label='fm')
axs.plot(aexp, fm2_200, label='fm2')
axs.set_xlabel('Scale Factor a = 1/(1+z)')
axs.set_ylabel(r'$\rho_s (DS_{a=1}, M/M_0)$')
axs.grid()
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.set_title('R500')
axs.plot(aexp, eta_500, label='eta')
axs.plot(aexp, delta_500, label='delta')
axs.plot(aexp, fm_500, label='fm')
axs.plot(aexp, fm2_500, label='fm2')
axs.set_xlabel('Scale Factor a = 1/(1+z)')
axs.set_ylabel(r'$\rho_s (DS_{a=1}, M/M_0)$')
axs.grid()
plt.legend()
plt.show()

In [ ]:
eta_200_p10 = datutils.get_perc(mah_ds_dict, param='eta_200[3]', q=10)
eta_200_p25 = datutils.get_perc(mah_ds_dict, param='eta_200[3]', q=25)
eta_200_p50 = datutils.get_perc(mah_ds_dict, param='eta_200[3]', q=50)
eta_200_p75 = datutils.get_perc(mah_ds_dict, param='eta_200[3]', q=75)
eta_200_p90 = datutils.get_perc(mah_ds_dict, param='eta_200[3]', q=90)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.set_title('eta_200')
axs.plot(aexp, eta_200_p50, color='b')
axs.fill_between(aexp, eta_200_p10, eta_200_p90, color='b', alpha=0.2)
axs.fill_between(aexp, eta_200_p25, eta_200_p75, color='b', alpha=0.3)
axs.set_xlabel('Scale Factor a = 1/(1+z)')
axs.set_ylabel(r'$\rho_s (DS_{a=1}, M/M_0)$')
axs.grid()
plt.show()

### Now for aexp(M/M0)...

In [ ]:
size = 100  
mm0 = np.linspace(1e-5, 1, size)      # set up mah array
mm0_dict = {}
for m in mm0:
    # create dataframe for each mm0 value
    mm0_dict[m] = pd.DataFrame(columns=['aexp'])

In [ ]:
tol = 1e-2
for mah in mah_df_list:
    # For each row in each mah snap, if it is close to an mm0 value then go to 
    # dataframe at corresponding mm0_dict key and add the row(s) to that df.
    # In the end, for each mm0 value there is a df of aexp values.
    for m in mm0: 
        err = np.abs((mah['M/M0'] - m))
        mask = err < tol
        mm0_dict[m] = pd.concat([mm0_dict[m], mah['aexp'][mask]])

In [ ]:
# now we assemble the dictionary of ds+aexp params
# each key is an mm0 val, each val is the joint df 
ds_aexp_dict = {}
for k, df in mm0_dict.items():
   if len(df) > len(dsdf):
      df1 = df.sample(len(dsdf))
      df2 = dsdf
   else:
      df1 = df
      df2 = dsdf.sample(len(dsdf))
   ds_aexp_dict[k] = pd.concat([df2.reset_index(drop=True),
                                df1.reset_index(drop=True)], axis=1)

In [ ]:
ds_aexp_corrs = {}
for k, v in ds_aexp_dict.items():
    ds_aexp_corrs[k] = v.corr(method='spearman')

In [ ]:
eta_200 = [df.loc['eta_200[3]', 'aexp'] for _, df in ds_aexp_corrs.items()]
delta_200 = [df.loc['delta_200[4]', 'aexp'] for _, df in ds_aexp_corrs.items()]
fm_200 = [df.loc['fm_200[5]', 'aexp'] for _, df in ds_aexp_corrs.items()]
fm2_200 = [df.loc['fm2_200[6]', 'aexp'] for _, df in ds_aexp_corrs.items()]
eta_500 = [df.loc['eta_500[8]', 'aexp'] for _, df in ds_aexp_corrs.items()]
delta_500 = [df.loc['delta_500[9]', 'aexp'] for _, df in ds_aexp_corrs.items()]
fm_500 = [df.loc['fm_500[10]', 'aexp'] for _, df in ds_aexp_corrs.items()]
fm2_500 = [df.loc['fm2_500[11]', 'aexp'] for _, df in ds_aexp_corrs.items()]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(16, 8))
fig.suptitle('R200')

axs[0, 0].plot(mm0, eta_200, label='eta')
axs[0, 0].grid()
axs[0, 0].legend()

axs[0, 1].plot(mm0, delta_200, label='delta')
axs[0, 1].grid()
axs[0, 1].legend()

axs[1, 0].plot(mm0, fm_200, label='fm')
axs[1, 0].grid()
axs[1, 0].legend()

axs[1, 1].plot(mm0, fm2_200, label='fm2')
axs[1, 1].grid()
axs[1, 1].legend()

axs[1, 0].set_xlabel('M/M0')
axs[1, 1].set_xlabel('M/M0')
axs[0, 0].set_ylabel(r'$\rho_s (DS_{a=1},aexp)$')
axs[1, 0].set_ylabel(r'$\rho_s (DS_{a=1},aexp)$')

plt.tight_layout()
plt.legend()
plt.show()

## DS only

In [ ]:
dsdf_list = []
for file in sorted(os.listdir('data/gadgetx3k_20/G3X_progenitors/')):
    ds_file = 'data/gadgetx3k_20/G3X_progenitors/' + file
    ds_df = pd.read_csv(ds_file, sep=r'\s+', index_col=False)
    dsdf_list.append(ds_df)

In [ ]:
ds_corr_list =[]
for df in dsdf_list:
    corr = df.corr(method='spearman')
    ds_corr_list.append(corr)

In [ ]:
redshift_list = pd.read_csv('data/gadgetx3k_20/redshift_list.txt', sep=r'\s+')
redshift_list[32:]  # DS files start at snap 32

In [ ]:
eta_delta200 = [df.loc['delta_200[4]', 'eta_200[3]'] for df in ds_corr_list]
eta_fm200 = [df.loc['fm_200[5]', 'eta_200[3]'] for df in ds_corr_list]
eta_fm2200 = [df.loc['fm2_200[6]', 'eta_200[3]'] for df in ds_corr_list]

eta_delta500 = [df.loc['delta_500[9]', 'eta_500[8]'] for df in ds_corr_list]
eta_fm500 = [df.loc['fm_500[10]', 'eta_500[8]'] for df in ds_corr_list]
eta_fm2500 = [df.loc['fm2_500[11]', 'eta_500[8]'] for df in ds_corr_list]

delta_eta200 = [df.loc['eta_200[3]', 'delta_200[4]'] for df in ds_corr_list]
delta_fm200 = [df.loc['fm_200[5]', 'delta_200[4]'] for df in ds_corr_list]
delta_fm2200 = [df.loc['fm2_200[6]', 'delta_200[4]'] for df in ds_corr_list]

delta_eta500 = [df.loc['eta_500[8]', 'delta_500[9]'] for df in ds_corr_list]
delta_fm500 = [df.loc['fm_500[10]', 'delta_500[9]'] for df in ds_corr_list]
delta_fm2500 = [df.loc['fm2_500[11]', 'delta_500[9]'] for df in ds_corr_list]

fm_eta200 = [df.loc['eta_200[3]', 'fm_200[5]'] for df in ds_corr_list]
fm_delta200 = [df.loc['delta_200[4]', 'fm_200[5]'] for df in ds_corr_list]
fm_fm2200 = [df.loc['fm2_200[6]', 'fm_200[5]'] for df in ds_corr_list]

fm_eta500 = [df.loc['eta_500[8]', 'fm_500[10]'] for df in ds_corr_list]
fm_delta500 = [df.loc['delta_500[9]', 'fm_500[10]'] for df in ds_corr_list]
fm_fm2500 = [df.loc['fm2_500[11]', 'fm_500[10]'] for df in ds_corr_list]

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(14, 20))

axs[0][0].plot(redshift_list[32:]['a'], eta_fm200, label='fm')
axs[0][0].plot(redshift_list[32:]['a'], eta_fm2200, label='fm2')
axs[0][0].plot(redshift_list[32:]['a'], eta_delta200, label='delta')
axs[0][0].set_ylabel(r'$\rho_s (eta, X)_{200}$')
axs[0][0].set_xlabel('aexp')
axs[0][0].grid()
axs[0][0].legend()

axs[0][1].plot(redshift_list[32:]['a'], eta_fm500, label='fm')
axs[0][1].plot(redshift_list[32:]['a'], eta_fm2500, label='fm2')
axs[0][1].plot(redshift_list[32:]['a'], eta_delta500, label='delta')
axs[0][1].set_ylabel(r'$\rho_s (eta, X)_{500}$')
axs[0][1].set_xlabel('aexp')
axs[0][1].grid()
axs[0][1].legend()

axs[1][0].plot(redshift_list[32:]['a'], delta_eta200, label='eta')
axs[1][0].plot(redshift_list[32:]['a'], delta_fm200, label='fm')
axs[1][0].plot(redshift_list[32:]['a'], delta_fm2200, label='fm2')
axs[1][0].set_ylabel(r'$\rho_s (delta, X)_{200}$')
axs[1][0].set_xlabel('aexp')
axs[1][0].grid()
axs[1][0].legend()

axs[1][1].plot(redshift_list[32:]['a'], delta_eta500, label='eta')
axs[1][1].plot(redshift_list[32:]['a'], delta_fm500, label='fm')
axs[1][1].plot(redshift_list[32:]['a'], delta_fm2500, label='fm2')
axs[1][1].set_ylabel(r'$\rho_s (delta, X)_{500}$')
axs[1][1].set_xlabel('aexp')
axs[1][1].grid()
axs[1][1].legend()

axs[2][0].plot(redshift_list[32:]['a'], fm_eta200, label='eta')
axs[2][0].plot(redshift_list[32:]['a'], fm_delta200, label='delta')
axs[2][0].plot(redshift_list[32:]['a'], fm_fm2200, label='fm2')
axs[2][0].set_ylabel(r'$\rho_s (fm, X)_{200}$')
axs[2][0].set_xlabel('aexp')
axs[2][0].grid()
axs[2][0].legend()

axs[2][1].plot(redshift_list[32:]['a'], fm_eta500, label='eta')
axs[2][1].plot(redshift_list[32:]['a'], fm_delta500, label='delta')
axs[2][1].plot(redshift_list[32:]['a'], fm_fm2500, label='fm2')
axs[2][1].set_ylabel(r'$\rho_s (eta, X)_{500}$')
axs[2][1].set_xlabel('aexp')
axs[2][1].grid()
axs[2][1].legend()

plt.legend()
plt.show()

## Heatmaps of DS(a=1) on Roan and Elena's files

In [ ]:
rds_today = 'data/gadgetx3k_20/G3X_progenitors/DS_G3X_snap_128_center-cluster_progenitors.txt'
rds_today = pd.read_csv(rds_today, sep=r'\s+')
rds_today.drop(labels=['rID[0]', 'Hid[1]', 'DS_200[2]', 'DS_500[7]'], 
               axis=1, inplace=True)
rds_today = rds_today[:20]

eds_today = 'data/gadgetx3k_20/snap_125.dyn'
eds_today = pd.read_csv(eds_today, sep=r'\s+', header=1)
eds_today.drop(labels=['region'], inplace=True, axis=1)

In [ ]:
display(rds_today)
display(eds_today)

In [ ]:
rds_corr = rds_today.corr(method='spearman')
plots.plot_corr_matrix(rds_corr)

In [ ]:
eds_corr = eds_today.corr(method='spearman')
plots.plot_corr_matrix(eds_corr)

## SM + MAH

In [ ]:
sm_df = pd.read_csv('results/rin_50kpc_rout_1Mpc.csv')
sm_df.columns

In [ ]:
sm_df.drop(columns=['Unnamed: 0', 'sky_mean', 'sky_median',
           'sky_sigma', 'flag', 'flag_sersic', 'flux_circ', 'flux_ellip', 
            'runtime', 'sn_per_pixel'], inplace=True)
sm_df.columns

In [ ]:
mah_df_list = []
for file in os.listdir('data/gadgetx3k_20/AHF_History/'):
    if '.dat' not in file:
        continue
    elif file == 'NewMDCLUSTER_0013_halo_128000000000001.dat':
            continue
    elif file == 'NewMDCLUSTER_0014_halo_128000000000001.dat':
            continue
    mah_file = 'data/gadgetx3k_20/AHF_History/' + file
    mah_df = pd.read_csv(mah_file, sep=r'\s+', index_col=False)
    mm0 = mah_df['Mvir(4)'].values/mah_df['Mvir(4)'][0]
    mm0 = pd.DataFrame(mm0)
    mm0.rename(columns={0: 'M/M0'}, inplace=True)
    mm0['Redshift'] = mah_df['Redshift(0)']
    mah_df_list.append(mm0)

In [ ]:
corrs_list = []
z_list = []
for z in mah_df_list[0]['Redshift']:
    if z <  0.06872:    #SM measurements done at snap 125
        continue
    mah_df = pd.DataFrame(columns=['M/M0'])

    for region in range(18):
        row = mah_df_list[region].loc[mah_df_list[region]
                                      ['Redshift'] == z, ['M/M0']]
        if not row.empty:
            mah_df = pd.concat([mah_df, row], ignore_index=True)

    if mah_df.shape[0] != sm_df.shape[0]:
        print(
            f"Skipping redshift {z}: mismatched rows {mah_df.shape[0]} vs {sm_df.shape[0]}")
        continue

    df = pd.concat([mah_df.reset_index(drop=True),
                   sm_df.reset_index(drop=True)], axis=1)
    corrs = df.corr(method='spearman')
    corrs_list.append(corrs)
    z_list.append(z)
z_array = np.array(z_list)

In [ ]:
aexp = 1/(1+z_array)
sersic_n = [df.loc['sersic_n', 'M/M0'] for df in corrs_list]
rhalf_ellip = [df.loc['rhalf_ellip', 'M/M0'] for df in corrs_list]
gini = [df.loc['Gini', 'M/M0'] for df in corrs_list]
rpetro_ellip = [df.loc['rpetro_ellip', 'M/M0'] for df in corrs_list]
orientation_asymmetry = [
    df.loc['orientation_asymmetry', 'M/M0'] for df in corrs_list]
rpetro_circ = [df.loc['rpetro_circ', 'M/M0'] for df in corrs_list]
rhalf_circ = [df.loc['rhalf_circ', 'M/M0'] for df in corrs_list]
m20 = [df.loc['M20', 'M/M0'] for df in corrs_list]              # stronger corr
fgm20 = [df.loc['F(G, M20)', 'M/M0'] for df in corrs_list]
sgm20 = [df.loc['S(G, M20)', 'M/M0'] for df in corrs_list]
conc = [df.loc['C', 'M/M0'] for df in corrs_list]
asym = [df.loc['A', 'M/M0'] for df in corrs_list]
smooth = [df.loc['S', 'M/M0'] for df in corrs_list]

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
axs.set_title('Statmorph Correlations')
axs.plot(aexp, sersic_n, label='sersic_n')
axs.plot(aexp, rhalf_ellip, label='r1/2')
axs.plot(aexp, gini, label='gini')
axs.set_xlabel('Scale Factor a = 1/(1+z)')
axs.set_ylabel(r'$\rho_s (SM_{a=0.94}, M/M_0)$')
axs.grid()
plt.legend()
plt.show()

## SM + DS

In [ ]:
# using new ICs
ds_df = pd.read_csv('data/gadgetx3k_20/snap_125_18.dyn', header=1, sep=r'\s+')
ds_df.drop(columns=['region'], inplace=True)
display(ds_df)
display(sm_df)

In [ ]:
df = pd.concat([ds_df, sm_df], axis=1)
df

In [ ]:
corrs = df.corr(method='spearman')
plots.plot_corr_matrix(corrs)

In [ ]:
# using old ICs

rds_today = 'data/gadgetx3k_20/G3X_progenitors/DS_G3X_snap_125_center-cluster_progenitors.txt'
rds_today = pd.read_csv(rds_today, sep=r'\s+')
rds_today.drop(columns=['rID[0]', 'Hid[1]', 'DS_200[2]', 'DS_500[7]'], 
               inplace=True)
rds_today = rds_today[:20]


In [ ]:
ds_df = rds_today.drop(labels=[14, 15], axis=0)
df = pd.concat([ds_df, sm_df], axis=1)
corrs = df.corr(method='spearman')
plots.plot_corr_matrix(corrs)

Now doing same as above but with annular mask...

In [ ]:
image = fits.open('data/gadgetx3k_20/maps/bcg_0001_125_0.fits')
image = image[0].data
center = (int(len(image[1])//2), int(len(image[0])//2))
segmap = imutils.annular_mask(image, center, r1=20, r2=1024)

In [ ]:
fig, axs = plt.subplots(1, 1)
plots.display_img(image, axs, mask=segmap)
plt.show()

In [ ]:
import astropy.units as u

In [ ]:
diameter = 5*u.Mpc
diameter.to(u.kpc).value + 50

In [ ]:
def real2pix(r, map=None, scale=5*u.Mpc):
    scale = scale.to(u.kpc)
    pixperkpc = 2048/scale.value
    radius = r.value*pixperkpc
    return int(radius)

In [ ]:
real2pix(1000*u.kpc)

In [ ]:
morphs_list = []
for id in np.arange(1, 21):
    try:
        file = fits.open(f'data/gadgetx3k_20/maps/bcg_{str(id).zfill(4)}_125_1.fits')
    except FileNotFoundError:
        continue
    print(f"Processing region {id}...")
    image = file[0].data
    center = (int(len(image[1])//2), int(len(image[0])//2))
    # r1 = 50*u.kpc
    # r1 = real2pix(r=r1, map=image)
    # r2 = 1*u.Mpc
    # r2 = real2pix(r=r2, map=image)
    segmap = imutils.annular_mask(image, center, r1=20, r2=409)

    fig, axs = plt.subplots(1, 1)
    plots.display_img(image, axs, mask=segmap)
    plt.show()
    morph = statmorph.source_morphology(image, segmap, gain=2.25)
    print(f'-------region {id} done-------')
    morphs_list.append(morph[0])

sm_df = datutils.create_morph_df(morphs_list, 
                                   name='results/rin_50kpc_rout_1Mpc.csv',
                                   save=True)